## Status code

200 -> A requisição deu certo

300 -> Redirecionada

400 -> Não conseguiu fazer a requisição

500 -> Deu um erro no sistema/url

## 🔹 Etapa 1

✅ Escolher uma API

✅ Fazer a extração de pelo menos 3 tabelas 


In [45]:
import requests
import pandas as pd
from numpy import dtype as np

# URLs da API
url_produto = 'https://fakestoreapi.com/products'
url_usuarios = 'https://fakestoreapi.com/users'
url_carrinhos = 'https://fakestoreapi.com/carts'

# Requisição para os produtos
res_produtos = requests.get(url_produto)
if res_produtos.status_code == 200:
    df_produtos = pd.DataFrame(res_produtos.json())
    print("Tabela de Produtos")
    print(df_produtos.head(5))
else:
    print(f"Erro ao acessar produtos: {res_produtos.status_code}")

# Requisição para os usuários
res_usuarios = requests.get(url_usuarios)
if res_usuarios.status_code == 200:
    df_usuarios = pd.DataFrame(res_usuarios.json())
    print("Tabela de Usuários")
    print(df_usuarios.head(5))
else:
    print(f"Erro ao acessar usuários: {res_usuarios.status_code}")

# Requisição para os carrinhos
res_carrinhos = requests.get(url_carrinhos)
if res_carrinhos.status_code == 200:
    df_carrinhos = pd.DataFrame(res_carrinhos.json())
    print("Tabela de Carrinhos")
    print(df_carrinhos.head(5))
else:
    print(f"Erro ao acessar carrinhos: {res_carrinhos.status_code}")



Tabela de Produtos
   id                                              title   price  \
0   1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1   2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2   3                                 Mens Cotton Jacket   55.99   
3   4                               Mens Casual Slim Fit   15.99   
4   5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   

                                         description        category  \
0  Your perfect pack for everyday use and walks i...  men's clothing   
1  Slim-fitting style, contrast raglan long sleev...  men's clothing   
2  great outerwear jackets for Spring/Autumn/Wint...  men's clothing   
3  The color could be slightly different between ...  men's clothing   
4  From our Legends Collection, the Naga was insp...        jewelery   

                                               image  \
0  https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...   
1  https://fakestoreapi.com

## 🔹 Etapa 2

✅ Realizar os tratamentos devidos das tabelas

In [46]:
df_produtos.rename(columns={'id': 'id_produto', 'title': 'nome_produto', 'price': 'preco', 'description': 'descricao', 'category': 'categoria', 'image': 'imagem', 'rating': 'avaliacao'}, inplace=True)
df_produtos['preco'] = df_produtos['preco'].astype(float)
df_produtos['id_produto'] = df_produtos['id_produto'].astype(int)
df_produtos['nome_produto'] = df_produtos['nome_produto'].astype(str)
df_produtos['descricao'] = df_produtos['descricao'].astype(str)
df_produtos['categoria'] = df_produtos['categoria'].astype(str)


df_produtos.head(5)

,id_produto,nome_produto,preco,descricao,categoria,imagem,avaliacao
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,"{'rate': 3.9, 'count': 120}"
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,"{'rate': 4.1, 'count': 259}"
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,"{'rate': 4.7, 'count': 500}"
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,"{'rate': 2.1, 'count': 430}"
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,"{'rate': 4.6, 'count': 400}"


In [47]:
print(df_produtos.dtypes)

id_produto        int64
nome_produto     object
preco           float64
descricao        object
categoria        object
imagem           object
avaliacao        object
dtype: object


In [ ]:
# Renomear as colunas para nomes mais legíveis
df_usuarios.rename(columns={
    'name': 'nome',
    'address': 'endereco',
    'id': 'id_produto',
    'email': 'e-mail',
    'username': 'usuario',
    'password': 'senha',
    'phone': 'telefone'
}, inplace=True)

# Criar coluna 'nome_completo' se a coluna 'nome' existir
if 'nome' in df_usuarios.columns:
    df_usuarios['nome_completo'] = df_usuarios['nome'].apply(
        lambda x: f"{x['firstname']} {x['lastname']}" if isinstance(x, dict) else ''
    )

# Extrair informações de endereço
if 'endereco' in df_usuarios.columns:
    # Extrair rua, cidade, número e CEP
    df_usuarios['rua'] = df_usuarios['endereco'].apply(
        lambda x: x['street'] if isinstance(x, dict) and 'street' in x else ''
    )
    df_usuarios['cidade'] = df_usuarios['endereco'].apply(
        lambda x: x['city'] if isinstance(x, dict) and 'city' in x else ''
    )
    df_usuarios['numero'] = df_usuarios['endereco'].apply(
        lambda x: x['number'] if isinstance(x, dict) and 'number' in x else ''
    )
    df_usuarios['cep'] = df_usuarios['endereco'].apply(
        lambda x: x['zipcode'] if isinstance(x, dict) and 'zipcode' in x else ''
    )

    # Extrair latitude e longitude
    df_usuarios['latitude'] = df_usuarios['endereco'].apply(
        lambda x: x['geolocation']['lat'] if isinstance(x, dict) and 'geolocation' in x else ''
    )
    df_usuarios['longitude'] = df_usuarios['endereco'].apply(
        lambda x: x['geolocation']['long'] if isinstance(x, dict) and 'geolocation' in x else ''
    )


    df_usuarios.drop(columns=['endereco'], inplace=True)







    
# Visualizar os primeiros registros
df_usuarios.head(5)


,id_produto,e-mail,usuario,senha,telefone,__v,nome_completo,rua,cidade,numero,cep,latitude,longitude
0,1,john@gmail.com,johnd,m38rmF$,1-570-236-7033,0,john doe,new road,kilcoole,7682,12926-3874,-37.3159,81.1496
1,2,morrison@gmail.com,mor_2314,83r5^_,1-570-236-7033,0,david morrison,Lovers Ln,kilcoole,7267,12926-3874,-37.3159,81.1496
2,3,kevin@gmail.com,kevinryan,kev02937@,1-567-094-1345,0,kevin ryan,Frances Ct,Cullman,86,29567-1452,40.3467,-30.1310
3,4,don@gmail.com,donero,ewedon,1-765-789-6734,0,don romer,Hunters Creek Dr,San Antonio,6454,98234-1734,50.3467,-20.1310
4,5,derek@gmail.com,derek,jklg*_56,1-956-001-1945,0,derek powell,adams St,san Antonio,245,80796-1234,40.3467,-40.1310
